In [13]:
import simpy
import numpy as np
import random

# define game and game components

In [174]:
class Trial(object):
    ''' One game play trial is defined by the proportion 
    of red/blue cards over the total number of cards and
    the opponent's shown outcome. '''
    
    n_cards = 5
    
    def __init__(self, trial_params):
        # blue card value = 1, red card value = -1
        self.n_red = trial_params[0]
        self.outcome = trial_params[1]
        self.n_blue = self.n_cards - self.n_red
        
        # create all playing cards
        self.cards = []
        self.cards.extend([-1] * self.n_red)
        self.cards.extend([1] * self.n_blue)
    
    def expectation(self) -> float:
        if self.outcome == -1:
            return self.outcome * self.n_red / self.n_cards
        if self.outcome == 1:
            return self.outcome * self.n_blue / self.n_cards
        else:
            raise ValueError("Opponent outcome has unacceptable value.")
    
    def selected_card(self):
        return random.choice(self.cards)

In [201]:
class Player(object):
    ''' Player (participant) attributes and possible 
    actions (lie or not lie) based on suspicion model
    and game rules. '''
    
    def __init__(self, player_params, pre_suspicion=None):
        #todo: alpha and baseline are to be estimated from actual playing behaviour, add equations
        self.alpha = player_params[0]
        self.baseline = player_params[1]

        # first trial has no pre_suspicion
        self.pre_suspicion = pre_suspicion if pre_suspicion is not None else 0
    
    def update_suspicion(self, value):
        self.pre_suspicion = value

In [216]:
def suspicion(pre_suspicion, alpha, baseline, outcome, expectation) -> float:
    '''Returns suspicion level according to hypothesized computational behaviour model'''
    return pre_suspicion + alpha * (outcome - expectation) + baseline

In [232]:
class Game(object):
    ''' Lets Player interact with given Trial(s)
    according to acquired suspicion level. '''
    
    def __init__(self, trials, player, n_trials=len(trials)):
        self.trials = trials # expected to be of class Trial
        self.player = player # expected to be of class Player
        self.n_trials = n_trials #todo: update self.trials if n_trials < or > len(trials)
    
    def get_and_update_suspicion(self):
        for index, t in enumerate(self.trials, start=1):
            print("trial: ", index, " has ", t.n_red, " red cards")
            
            new_suspicion = suspicion(self.player.pre_suspicion, self.player.alpha, self.player.baseline, t.outcome, t.expectation())
            print("suspicion at trial #", index, "is", new_suspicion)
            self.player.update_suspicion(new_suspicion)
            
    def play(self):
        for index, t in enumerate(self.trials, start=1):
            print("trial: ", index, " has ", t.n_red, " red cards")
            
            new_suspicion = suspicion(self.player.pre_suspicion, self.player.alpha, self.player.baseline, t.outcome, t.expectation())
            selected_card = t.selected_card()
            lied_card = player_lie(selected_card)
            
            if self.player.alpha * (t.outcome - t.expectation()) > self.player.baseline + self.player.pre_suspicion:
                print("player lies: ", lied_card, "\n")
            else:
                print("player plays selected card: ", selected_card, "\n")
            
            self.player.update_suspicion(new_suspicion)
    
    def player_lie(selected_card):
        # assumes that selected_card takes either value 1 or -1
        return 1 if selected_card == -1 else -1

# create game trials

In [78]:
# game play values
n_red = range(0, 6, 1)
outcome = (1, -1)

# all possible trial parameters
from itertools import product
trial_params = (list(product(n_red, outcome)))

In [124]:
trial_params

[(0, 1),
 (0, -1),
 (1, 1),
 (1, -1),
 (2, 1),
 (2, -1),
 (3, 1),
 (3, -1),
 (4, 1),
 (4, -1),
 (5, 1),
 (5, -1)]

In [80]:
# possible number of unique trials (= use as minimum?)
len(trial_params)

12

In [181]:
# create all game trials
trials = []

for params in trial_params:
    trials.append(Trial(params))

In [149]:
# check trial object parameter settings
trials[11].cards

[-1, -1, -1, -1, -1]

In [150]:
# full range of potential expectation violation
[t.outcome - t.expectation() for t in trials]

[0.0,
 -1.0,
 0.19999999999999996,
 -0.8,
 0.4,
 -0.6,
 0.6,
 -0.4,
 0.8,
 -0.19999999999999996,
 1.0,
 0.0]

# create game players

In [111]:
# list player attributes
alpha = np.linspace(0,1,11)
baseline = [-1, -0.5, -0.1, 0, 0.1, 0.5, 1]

player_params = (list(product(alpha, baseline)))

In [112]:
len(player_params)

77

In [136]:
# create Players according to above attributes
players = []
for params in player_params:
    players.append(Player(params))

In [135]:
# check some attributes
players[1].pre_suspicion

5

In [134]:
players[1].update_suspicion(5)

# let's play

In [248]:
for player in players:
    Game(trials, player).get_and_update_suspicion()

suspicion at trial # 1 is -49.0
suspicion at trial # 2 is -50.0
suspicion at trial # 3 is -51.0
suspicion at trial # 4 is -52.0
suspicion at trial # 5 is -53.0
suspicion at trial # 6 is -54.0
suspicion at trial # 7 is -55.0
suspicion at trial # 8 is -56.0
suspicion at trial # 9 is -57.0
suspicion at trial # 10 is -58.0
suspicion at trial # 11 is -59.0
suspicion at trial # 12 is -60.0
suspicion at trial # 1 is -6.5
suspicion at trial # 2 is -7.0
suspicion at trial # 3 is -7.5
suspicion at trial # 4 is -8.0
suspicion at trial # 5 is -8.5
suspicion at trial # 6 is -9.0
suspicion at trial # 7 is -9.5
suspicion at trial # 8 is -10.0
suspicion at trial # 9 is -10.5
suspicion at trial # 10 is -11.0
suspicion at trial # 11 is -11.5
suspicion at trial # 12 is -12.0
suspicion at trial # 1 is -1.3
suspicion at trial # 2 is -1.4000000000000001
suspicion at trial # 3 is -1.5000000000000002
suspicion at trial # 4 is -1.6000000000000003
suspicion at trial # 5 is -1.7000000000000004
suspicion at trial

suspicion at trial # 7 is -4.459999999999999
suspicion at trial # 8 is -5.279999999999999
suspicion at trial # 9 is -5.139999999999999
suspicion at trial # 10 is -5.799999999999999
suspicion at trial # 11 is -5.499999999999999
suspicion at trial # 12 is -5.999999999999999
suspicion at trial # 1 is -0.1
suspicion at trial # 2 is -1.0
suspicion at trial # 3 is -0.9400000000000001
suspicion at trial # 4 is -1.6800000000000002
suspicion at trial # 5 is -1.4600000000000002
suspicion at trial # 6 is -2.04
suspicion at trial # 7 is -1.6600000000000001
suspicion at trial # 8 is -2.08
suspicion at trial # 9 is -1.54
suspicion at trial # 10 is -1.8
suspicion at trial # 11 is -1.1
suspicion at trial # 12 is -1.2000000000000002
suspicion at trial # 1 is 0.0
suspicion at trial # 2 is -0.8
suspicion at trial # 3 is -0.6400000000000001
suspicion at trial # 4 is -1.2800000000000002
suspicion at trial # 5 is -0.9600000000000002
suspicion at trial # 6 is -1.4400000000000002
suspicion at trial # 7 is -0.

In [246]:
test = Game(trials, players[3])

In [247]:
test.player.pre_suspicion

0

In [244]:
test.get_and_update_suspicion()

suspicion at trial # 1 is -0.1
suspicion at trial # 2 is -0.2
suspicion at trial # 3 is -0.30000000000000004
suspicion at trial # 4 is -0.4
suspicion at trial # 5 is -0.5
suspicion at trial # 6 is -0.6
suspicion at trial # 7 is -0.7
suspicion at trial # 8 is -0.7999999999999999
suspicion at trial # 9 is -0.8999999999999999
suspicion at trial # 10 is -0.9999999999999999
suspicion at trial # 11 is -1.0999999999999999
suspicion at trial # 12 is -1.2


In [198]:
player_lie(trials[0].selected_card())

-1

# visualize results